# 영상 분할
### 1. 컨투어

In [11]:
### 컨투어 찾기와 그리기 = 연속된 색상이나 밝기의 곡선
import cv2
import numpy as np

# 이미지 불러오기
img = cv2.imread('./picture/shapes.png')
img2 = img.copy()

# 이미지 그레이 스케일로 변환
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# 스레시홀드로 바이너리 이미지 만들어서 검은색 배경에 흰색 물체 그림으로 변경
ret, imthres = cv2.threshold(imgray, 127, 255, cv2.THRESH_BINARY_INV)

# 바깥쪽 컨투어에 대해 모든 좌표 반환
contour, hierarchy = cv2.findContours(imthres, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
# 바깥쪽 컨투어에 대해 꼭짓점 좌표 반환
contour2, hierarchy = cv2.findContours(imthres, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

print('도형의 개수: %d(%d)' %(len(contour), len(contour2)))

# 컨투어 그리기 - 바깥쪽 컨투어
cv2.drawContours(img, contour, -1, (0, 255, 0), 4)
cv2.drawContours(img2, contour2, -1, (0, 255, 0), 4)

# 컨투어의 모든 점을 파랑색으로 그리기
for i in contour:
    for j in i:
        cv2.circle(img, tuple(j[0]), 1, (255, 0, 0), -1)

# 컨투어의 모든 점을 파랑색으로 그리기   
for i in contour2:
    for j in i:
        cv2.circle(img2, tuple(j[0]), 1, (255, 0, 0), -1)

cv2.imshow('CHAIN_APPROX_NONE', img)
cv2.imshow('CHAIN_APPROX_SIMPLE', img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

도형의 개수: 3(3)


In [16]:
### 컨투어 계층 트리
import cv2
import numpy as np

# 이미지 불러오기
img = cv2.imread('./picture/shapes_donut.png')
img2 = img.copy()

# 흑백이미지 → 바이너리 이미지로 변환하기
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, imthres = cv2.threshold(imgray, 127, 255, cv2.THRESH_BINARY_INV)

# 가장 바깥 컨투어만 수집
contour, hierarchy = cv2.findContours(imthres, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
print(len(contour), hierachy)

# 모든 컨투어를 트리 계층으로 수집
contour2, hierachy = cv2.findContours(imthres, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
print(len(contour2), hierarchy)

# 가장 바깥의 컨투어만 그리기
cv2.drawContours(img, contour, -1, (0, 255, 0), 3)

# 모든 컨투어 그리기
for idx, cont in enumerate(contour2):
    # 색상을 랜덤으로 고르기
    color = [int(i) for i in np.random.randint(0, 255, 3)]
    # 고른 색상으로 각 컨투어(영역) 그리기
    cv2.drawContours(img2, contour2, idx, color, 3)
    # 컨투어마다 이름 붙여주기
    cv2.putText(img2, str(idx,), tuple(cont[0][0]), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255))

cv2.imshow('RETR_EXTERNAL', img)
cv2.imshow('RETR_TREE', img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

3 [[[ 2 -1  1 -1]
  [-1 -1 -1  0]
  [ 4  0  3 -1]
  [-1 -1 -1  2]
  [-1  2  5 -1]
  [-1 -1 -1  4]]]
6 [[[ 1 -1 -1 -1]
  [ 2  0 -1 -1]
  [-1  1 -1 -1]]]


In [18]:
### 모멘트를 이용한 중심점, 넓이, 둘레 길이
import numpy as np
import cv2

# 이미지 불러오기
img = cv2.imread('./picture/shapes.png')
# 이미지 gray로 바꾸기
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# 이미지 바이너리 스케일로 변환하기
ret, th = cv2.threshold(imgray, 127, 255, cv2.THRESH_BINARY_INV)
# 이미지 컨투어 찾기
contours, hierachy = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 각 도형에 대한 컨투어 계산
for c in contours:
    # 각 도형별 모멘트 계산
    mmt = cv2.moments(c)
    # 중심 모멘트 계산하기 = 위치가 변해도 값이 동일한 모멘트
    cx = int(mmt['m10']/mmt['m00'])
    cy = int(mmt['m01']/mmt['m00'])
    # 영역 넓이
    a = mmt['m00']
    # 영역 외곽선 길이 구하기: arcLength(넓이를 계산할 컨투어, 컨투어 방향 플래그)
    l = cv2.arcLength(c, True)
    # 중심점에 노란색 점 그리기
    cv2.circle(img, (cx, cy), 5, (0, 255, 255), -1)
    # 중심점 아래에 넓이 쓰기 - 빨강
    cv2.putText(img, "A:%.0f"%a, (cx, cy+20), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255))
    # 컨투어 시작점에 길이 적기 - 파랑
    cv2.putText(img, "L:%.2f"%l, tuple(c[0][0]), cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 0))
    # 함수로 컨투어 넓이 계산해서 출력
    print("area:%.2f"%cv2.contourArea(c, False))

# 결과 출력하기
cv2.imshow('center', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

area:9870.00
area:12544.00
area:6216.00


In [29]:
### 컨투어를 감싸는 도형 그리기
import numpy as np
import cv2

# 이미지 읽어오기
img = cv2.imread('./picture/lightning.png')
# 그레이스케일로 변환
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# 바이너리 스케일 변환
ret, th = cv2.threshold(imgray, 127, 255, cv2.THRESH_BINARY_INV)

# 컨투어 찾기
contours, hr = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

contr = contours[0]

# ① 감싸는 사각형 표시(검정) = boundingRect(좌표를 감싸는 사각형 구하기)
x, y, w, h = cv2.boundingRect(contr)
cv2.rectangle(img, (x,y), (x+w, y+h), (0, 0, 0), 3)

# ② 좌표를 감싸는 최소한의 사각형 (초록색) = miniAreaRect(회전한 사각형의 좌표?)
rect = cv2.minAreaRect(contr)
# 중심점과 각도를 4개의 꼭짓점 좌표로 변환
box = cv2.boxPoints(rect)
# 정수로 변환
box = np.int0(box)
# 컨투어 그리기
cv2.drawContours(img, [box], -1, (0,255,0), 3)

# ③ 최소한의 원 표시(파란색)
(x, y), radius = cv2.minEnclosingCircle(contr)
cv2.circle(img, (int(x), int(y)), int(radius), (255, 0, 0), 2)

# ④ 감싸는 최소한의 삼각형 표시 (분홍색)
ret, tri = cv2.minEnclosingTriangle(contr)
cv2.polylines(img, [np.int32(tri)], True, (255, 0, 255),  2)

# ⑤ 최소한의 타원 표시 (노란색)
ellipse = cv2.fitEllipse(contr)
cv2.ellipse(img, ellipse, (0,255,255), 3)

# ⑥ 중심점을 통과하는 직선 표시 (빨간색) = fitLine(): 중심점을 통과하는 직선 계산, 거리 계산방식, [반지름 정확도, 각도 정확도, line] 이셋은 권장값대로
[vx, vy, x, y] = cv2.fitLine(contr, cv2.DIST_L2, 0, 0.01, 0.01)
cols, rows = img.shape[:2]
cv2.line(img, (0, int((0-x)*(vy/vx)+y)), (cols-1, int((cols-x)*(vy/vx)+y)), (0,0,255), 2)

cv2.imshow('Bound Fit shapes', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [34]:
### 컨투어 단순화 = 오차 범위 내 근사 값으로 컨투어를 계산
import cv2
import numpy as np

# 이미지 불러오기
img = cv2.imread('./picture/bad_rect.png')
img2 = img.copy()

# 이미지 grayscale로 변환
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# 이미지 바이너리 스케일로 변환
ret, th = cv2.threshold(imgray, 127, 255, cv2.THRESH_BINARY)

# 컨투어 찾기
contours, hierachy = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contour = contours[0]

# 전체 둘레 길이 * 0.05
epsilon = 0.05 * cv2.arcLength(contour, True)
# 근사 컨투어 계산 = approxPolyDP(대상 컨투어, 근사값의 정확도, 컨투어의 닫힘 여부)
approx = cv2.approxPolyDP(contour, epsilon, True)

# 컨투어 그리기
cv2.drawContours(img, [contour], -1, (0, 255, 0), 3)
cv2.drawContours(img2, [approx], -1, (0, 255, 0), 3)

cv2.imshow('contour', img)
cv2.imshow('approx', img2)
cv2.waitKey()
cv2.destroyAllWindows()

In [37]:
### 볼록 선체 = 객체의 외곽 선체 찾기에 도움
import numpy as np
import cv2

# 이미지 불러오기
img = cv2.imread('./picture/hand.jpg')
img2 = img.copy()
# 이미지 그래이스케일
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# 이미지 바이너리 스케일로 변환
ret, th = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)

# ① 컨투어 찾기와 그리기
contours, hierachy = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cntr = contours[0]
# 컨투어 그리기
cv2.drawContours(img, [cntr], -1, (0, 255, 0), 1)

# ② 볼록 선체 찾기 = convexHull(입력 컨투어, [볼록선체 결과, 방향, 결과 좌표 형식 선택])
hull = cv2.convexHull(cntr)
# 볼록 선체 컨투어 그리기
cv2.drawContours(img2, [hull], -1, (0, 255, 0), 1)
# 볼록 선체 만족 여부 확인 = 일반 컨투어 vs 볼록 선체
print(cv2.isContourConvex(cntr), cv2.isContourConvex(hull))

# 볼록 선체 찾기(인덱스로?)
hull2 = cv2.convexHull(cntr, returnPoints=False)
# 볼록선체의 결함찾기
defects = cv2.convexityDefects(cntr, hull2)

# 볼록선체 결함 순회
for i in range(defects.shape[0]):
    # 오목점 시작, 끝, 가장 먼거리, 볼록 선체와의 거리
    startP, endP, farthestP, distance = defects[i, 0]
    # 가장 먼 지점의 좌표 구하기
    farthest = tuple(cntr[farthestP][0])
    # 거리를 부동 소수점으로 변환
    dist = distance/256.0
    # 거리가 1보다 큰 경우 빨간점 표시
    if dist > 1:
        cv2.circle(img2, farthest, 3, (0, 0, 255), -1)

cv2.imshow('contour', img)
cv2.imshow('convex hull', img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

False True


In [40]:
### 도형 매칭으로 비슷한 도형 찾기
import numpy as np
import cv2

# 이미지 불러오기
target = cv2.imread('./picture/4star.jpg')
shapes = cv2.imread('./picture/shapestomatch.jpg')

# 이미지 그레이 스케일로 전부 변환
targetGray = cv2.cvtColor(target, cv2.COLOR_BGR2GRAY)
shapesGray = cv2.cvtColor(shapes, cv2.COLOR_BGR2GRAY)

# 이미지 바이너리 스케일로 변환 → 이미지 하양,  배경 검정
ret, targetTh = cv2.threshold(targetGray, 127, 255, cv2.THRESH_BINARY_INV)
ret, shapesTh = cv2.threshold(shapesGray, 127, 255, cv2.THRESH_BINARY_INV)

# 하얀 이미지에서 컨투어 찾기
cntrs_target, _ = cv2.findContours(targetTh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cntrs_shapes, _ = cv2.findContours(shapesTh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 각 도형과 매칭을 위한 반복문
# 컨트어와 매칭 점수를 보관할 리스트
matchs = []
for contr in cntrs_shapes:
    # 두개의 컨투어로 도형 매칭하여 닮음 정도를 보여줌 → cv2.matchShapes(컨투어1 컨투어2, 비교 알고리즘)
    match = cv2.matchShapes(cntrs_target[0], contr, cv2.CONTOURS_MATCH_I2, 0.0) # 목록 중 하나를 꺼내서 target과 비교
    # 그 결과 생긴 contr와 match 점수 붙이기
    matchs.append((match, contr))
    # match 점수를 적기
    cv2.putText(shapes, '%.2f'%match, tuple(contr[0][0]), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 1)

# matchs를 매칭 점수 순으로 정렬
matchs.sort(key=lambda x: x[0])
# 가장 적은 매칭 점수를 얻는 도형 컨투어에 선 그리기
cv2.drawContours(shapes, [matchs[0][1]], -1, (0, 255, 0), 3)
cv2.imshow('target', target)
cv2.imshow('Match Shape', shapes)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 2. 허프 변환

In [42]:
### 허프 선 변환 = 수많은 픽셀 속에서 직선 관계를 갖는 픽셀들만 골라내기
import numpy as np
import cv2

# 이미지 불러오기
img = cv2.imread('./picture/sudoku.jpg')
img2 = img.copy()
# 이미지의 가로 세로 설정
h, w = img.shape[:2]
# 이미지 그레이 스케일 변환
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# 케니로 엣지 검출
edges = cv2.Canny(imgray, 100, 200)

# 허프선(여러 점들과 원점이 직각이 되는 선의 각도와 길이가 같은 선)의 검출 = HoughLines(이미지, 거리 측정 헤상도, 라디안 단위, 직선으로 판단할 최소한의 동일 픽셀수)
lines = cv2.HoughLines(edges, 1, np.pi/180, 130)
for line in lines:
    # 이렇게 만들어진 선에서 거리와 각도 구하기
    r, theta = line[0]
    # x, y 축에 대한 삼각비 구하기 = 기울기로 사용이 가능...?
    tx, ty = np.cos(theta), np.sin(theta)
    # x, y 절편 좌표
    x0, y0 = tx*r, ty*r
    # 두 기준 좌료에 빨간색 점 그리기
    cv2.circle(img2, (int(abs(x0)), int(abs(y0))), 3, (0,0,255), -1)
    # 직선 방정식으로 그리기 위한 끝점과 시작점 계산
    x1, y1 = int(x0 + w*(-ty)), int(y0 + h * tx)
    x2, y2 = int(x0 - w*(-ty)), int(y0 - h * tx)
    # 선그리기
    cv2.line(img2, (x1, y1), (x2, y2), (0, 255, 0), 1)

# 결과의 출력
merged = np.hstack((img, img2))
cv2.imshow('hough line', merged)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [43]:
# 확률적 허프 선 변환 = 모든 점에 대해서가 아니라 무작위로 선정한 픽셀로 허프변환을 수행하여 그 수를 늘려가는 방법
import cv2
import numpy as np

# 이미지 불러오기
img = cv2.imread('./picture/sudoku.jpg')
img2 = img.copy()

# 그래이 스케일 변환
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# 엣지 검출
edges = cv2.Canny(imgray, 50, 200)

# 확률 허프 변환 적용 = HoughLinesP(엣지, 거리 측정 해상도, 라디안 단위, 직선으로 판단할 최소한의 동일 픽셀수, 선으로 인정할 최소 길이, 선으로 판단한 최대간격, 검출된 선 좌표)
lines = cv2.HoughLinesP(edges, 1, np.pi/180, 10, None, 20, 2)
for line in lines:
    # 검출된 선 그리기
    x1, y1, x2, y2 = line[0]
    cv2.line(img, (x1, y1), (x2, y2), (0, 255, 0), 1)

cv2.imshow('Probability hough line', img)
cv2.waitKey()
cv2.destroyAllWindows()

In [50]:
### 허프 원 변환
# 방법1. 허프 직선 변환 알고리즘 적용 (메모리와 연산속도 문제)
# 방법2. 캐니 엣지 수행 → 소벨 필터 적용 : 엣지의 경사도를 누적
import cv2
import numpy as np

# 이미지 불러오기
img = cv2.imread('./picture/coins_connected.jpg')
# 그레이 스케일 변환
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# 노이즈 제거를 위한 가우시안 블러
blur = cv2.GaussianBlur(gray, (3, 3), 0)
# 하프 원 변환 적용 = HouchCircles(이미지, 검출 방식 선택, 입력 영상과 경사 누적의 해상도 반비례율, 원들 중심간의 최소거리, 원 검출 결과, 캐니 엣지 스레시홀드 최대값)
circles = cv2.HoughCircles(blur, cv2.HOUGH_GRADIENT, 1.5, 30, None, 300)
# 만약 원이 존재하면
if circles is not None:
    circles = np.uint16(np.around(circles))
    for i in circles[0,:]:
        # 원 둘레에 초록색 원 그리기
        cv2.circle(img, (i[0], i[1]), i[2], (0, 255, 0), 2)
        # 원의 중심점에 빨간색 원 그리기
        cv2.circle(img, (i[0], i[1]), 2, (0, 0, 255), 5)

cv2.imshow('hough circle', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 3. 연속 영역 분할

In [51]:
# 거리 변환 = 물체의 최중심점을 찾기 = 스켈레톤 검출 → 방법1. 주변 경계로부터 가장 멀리 떨어진 곳 찾기
import cv2
import numpy as np

# 이미지 그레이스케일로 불러오기
img = cv2.imread('./picture/full_body.jpg', cv2.IMREAD_GRAYSCALE)
# 바이너리 스케일로 변환하기 = 이미지는 하양, 배경은 검정
_, biimg = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)

# 거리 변환 = distanceTransform(입력 영상, 거리계산 방식, 거리 변환 커널의 크기)
dst = cv2.distanceTransform(biimg, cv2.DIST_L2, 5)
# 거리 값을 정규화 (0~255)
dst = (dst/(dst.max() - dst.min()) * 255).astype(np.uint8)
# 거리 값에 스레시홀드로 완전한 뼈대 찾기 = 영역별로 국소적인 thresholding 계산을 하기
skeleton = cv2.adaptiveThreshold(dst, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 7, -3)

cv2.imshow('original', img)
cv2.imshow('dist', dst)
cv2.imshow('skel', skeleton)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [59]:
### 연결 요소 레이블링
import cv2
import numpy as np

# 이미지 불러오기
img = cv2.imread('./picture/shapes_donut.png')
# 이미지와 똑같은 모양의 img2 만들기
img2 = np.zeros_like(img)
# 이미지를 그레이 스케일로 변환
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# 이미지를 바이너리 스케일로 변환하기
_, th = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

# 이미지에서 픽셀값이 0으로 끊어지지 않는 영역끼리 같은 값을 부여
cnt, labels = cv2.connectedComponents(th)

# 레이블 개수만큼 순회
for i in range(cnt):
    # 레이블이 같은 영역에 랜"덤한 색상 적용
    img2[labels==i] = [int(j) for j in np.random.randint(0, 255, 3)]

cv2.imshow('origin', img)
cv2.imshow('labeled', img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [70]:
### 색 채우기 : 연속되는 영역에 같은 색상 채워 넣기
import numpy as np
import cv2

# 이미지 불러오기
img = cv2.imread('./picture/taekwonv1.jpg')
# 이미지의 가로, 세로 지정
rows, cols = img.shape[:2]
# 이미지와 같은 크기의 zero 마스크를 만든다. 이때 2씩 더해서 상하좌우 1씩 여백을 만든다.
mask = np.zeros((rows+2, cols+2), np.uint8)
# 채우기에 사용할 색(흰색) 지정
newVal = (255, 255, 255)
# 최소, 최대 차이값 = 이정도 차이는 같은 색으로 인식
loDiff, upDiff = (7, 7, 7), (7, 7, 7)

# 마우스 이벤트 처리 함수
def onMouse(event, x, y, flags, param):
    global mask, img
    # 만약 클릭하면
    if event == cv2.EVENT_LBUTTONDOWN:
        seed = (x, y)
        # 색 채우기 적용
        retval = cv2.floodFill(img, mask, seed, newVal, loDiff, upDiff)
        # 채운 뒤의 변경 결과 표시
        cv2.imshow('img', img)

# 화면 출력
cv2.imshow('img', img)
cv2.setMouseCallback('img', onMouse)
cv2.waitKey()
cv2.destroyAllWindows()

In [72]:
### 워터셰드 = 연속한 영역 찾기
import cv2
import numpy as np

# 이미지 불러오기
img = cv2.imread('./picture/taekwonv1.jpg')
rows, cols = img.shape[:2]
img_draw = img.copy()

# 이미지와 같은 크기의 마커 생성
marker = np.zeros((rows, cols), np.int32)
# 마커 아이디 1에서 시작
markerId = 1
# 선택한 색상 값을 저장할 공간
colors = []
# 드래그 여부 확인 변수
isDragging = False

# 마우스 이벤트 처리 함수
def onMouse(event, x, y, flags, param):
    global img_draw, marker, markerId, isDragging
    
    # 마우스 클릭
    if event == cv2.EVENT_LBUTTONDOWN:
        # 드래그 시작 & 진행중
        isDragging = True
        # 각 시작점 좌표의 색상 선택
        colors.append((markerId, img[y,x]))
    
    # 마우스 움직임
    elif event == cv2.EVENT_MOUSEMOVE:
        # 드래그 진행중
        if isDragging:
            # 마우스 좌표에 해당하는 마커의 좌표를 아이디 값으로 채우기
            marker[y, x] = markerId
            # 마커 표시한 곳에 붉은 원 그리기
            cv2.circle(img_draw, (x,y), 3, (0,0,255), -1)
            cv2.imshow('watershed', img_draw)

    # 클릭에서 손을 뗀 경우 
    elif event == cv2.EVENT_LBUTTONUP:
        if isDragging:
            # 드래그 상태는 꺼지고
            isDragging = False
            # 마커 아이디에 1을 더한다.
            markerId += 1

    # 만약 마우스 우클릭 이벤트가 발생한 경우
    elif event == cv2.EVENT_RBUTTONDOWN:
        # 워터 셰드 적용으로 마커 아이디에 따라 이미지 분리가 일어난다.
        cv2.watershed(img, marker)
        # 이때 마커에 -1로 표현된 부분(경계)를 녹색으로 표시
        img_draw[marker==-1] = (0, 255, 0)
        # 선택한 마커 아이디만큼
        for mid, color in colors:
            # 각 이미지의 영역을 생상으로 채우기
            img_draw[marker==mid] = color
        cv2.imshow('watershed', img_draw)

cv2.imshow('watershed', img)
cv2.setMouseCallback('watershed', onMouse)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
### 그랩컷 = 전경과 객체를 분리해준다.
import cv2
import numpy as np

# 이미지 불러오기
img = cv2.imread('./picture/taekwonv1.jpg')
img_draw = img.copy()
# 마스크 생성
mask = np.zeros(img.shape[:2], dtype=np.uint8)
# 사각형 영역 좌표 초기화
rect = [0, 0, 0, 0]
# 그랩컷의 초기 모드 = 재시도
mode = cv2.GC_EVAL

# 함수 내에서 사용할 배경과 전경 모델 임시 배열 버퍼
bgdmodel = np.zeros((1, 65), np.float64)
fgdmodel = np.zeros((1, 65), np.float64)

# 마우스 이벤트 처리 함수
def onMouse(event, x, y, flags, param):
    global mouse_mode, rect, mask, mode
    
    # 마우스 왼쪽 클릭하면
    if event == cv2.EVENT_LBUTTONDOWN:
        # 아무 키도 눌리지 않은 경우
        if flags <= 1:
            # 드래그 시작 ( 사각형 그리기 모드 )
            mode = cv2.GC_INIT_WITH_RECT
            # 시작 좌표를 x, y로 저장
            rect[:2] = x, y

    # 만약 마우스 왼쪽 누른 상태에서 움직이면        
    elif event == cv2.EVENT_MOUSEMOVE and flags & cv2.EVENT_FLAG_LBUTTON:
        # 사각형 그리기 모드로 드래그 진행중이면
        if mode == cv2.GC_INIT_WITH_RECT:
            # 그리고 있는 이미지 복사해서 그 위에 사각형 그리는걸 보여주기
            img_temp = img.copy()
            cv2.rectangle(img_temp, (rect[0], rect[1]), (x, y), (0,255,0), 2)
            cv2.imshow('img', img_temp)

        # 만약 키가 눌려있다면 마스크 모드 시작!
        elif flags > 1:
            mode = cv2.GC_INIT_WITH_MASK
            
            # ctrl 키가 눌린 경우
            if flags & cv2.EVENT_FLAG_CTRLKEY:
                # 흰색 점 그리기
                cv2.circle(img_draw, (x, y), 3, (255, 255, 255), -1)
                # 클릭한 좌표를 확실한 전경으로 선택
                cv2.circle(mask, (x, y), 3, cv2.GC_FGD, -1)
            # 쉬프트키가 눌린경우
            if flags & cv2.EVENT_FLAG_SHIFTKEY:
                # 검정색 점 그리기
                cv2.circle(img_draw, (x, y), 3, (0,0,0), -1)
                # 클릭한 좌표를 확실한 배경으로 선택
                cv2.circle(mask, (x, y), 3, cv2.GC_BGD, -1)
            cv2.imshow('img', img_draw)

    # 마우스 오른쪽 버튼에서 떼면
    elif event == cv2.EVENT_LBUTTONUP:
        # 사각형 그리는 모드중이면
        if mode == cv2.GC_INIT_WITH_RECT:
            # x, y 좌표 수집
            rect[2:] = x, y
            # 사각형을 화면에 그려서 출력하기
            cv2.rectangle(img_draw, (rect[0], rect[1]), (x, y), (255,0,0), 2)
            cv2.imshow('img', img_draw)
        
        # 그랩컷 적용 : grabCut(입력 영상, 입력영상과 크기가 같은 배열로 배경과 전경을 구분하는 값을 가지고 있음, 전경의 사각형 좌표, 임시 배열 버퍼, 반복횟수, 동장 방법)
        cv2.grabCut(img, mask, tuple(rect), bgdmodel, fgdmodel, 1, mode)
        img2 = img.copy()
        # 마스크에 확실한 배경(GC_BGD), 또는 아마도 배경(GC_PB_BGD)를 0으로 채우기
        img2[(mask==cv2.GC_BGD) | (mask==cv2.GC_PR_BGD)] = 0
        # 최종 결과 출력
        cv2.imshow('grabcut', img2)
        # 모드 리셋
        mode = cv2.GC_EVAL

# 초기화면 출력
cv2.imshow('img', img)
# 마우스 이벤트 등록
cv2.setMouseCallback('img', onMouse)
# esc 누를때까지 계속 작동하기
while True:
    if cv2.waitKey(0) & 0xFF == 27:
        break
cv2.destroyAllWindows()

In [2]:
# 평균 이동 필터 (세그멘테이션) = 일정 반경의 원 내의 평균 값을 중심으로 바꿔서 이동하면서 가장 밀접한 곳 찾기
# 이동을 시작한 지점 ~ 이동 중지한 지점 = 하나의 영역으로 연결된 지점 찾기
import numpy as np
import cv2

# 이미지 불러오기
img = cv2.imread('./picture/taekwonv1.jpg')

# 트랙바 이베트 처리 함수
def onChange(x):
    # sp : 공간 윈도 반지름 크기
    sp = cv2.getTrackbarPos('sp', 'img')
    # sr : 색상 윈도 반지름 크기
    sr = cv2.getTrackbarPos('sr', 'img')
    # lv : 
    lv = cv2.getTrackbarPos('lv', 'img')

    mean = cv2.pyrMeanShiftFiltering(img, sp, sr, None, lv)
    cv2.imshow('img', np.hstack((img, mean)))

cv2.imshow('img', np.hstack((img, img)))
cv2.createTrackbar('sp', 'img', 0, 100, onChange)
cv2.createTrackbar('sr', 'img', 0, 100, onChange)
cv2.createTrackbar('lv', 'img', 0, 5, onChange)
cv2.waitKey()
cv2.destroyAllWindows()

[ WARN:0] global /tmp/pip-req-build-afu9cjzs/opencv/modules/highgui/src/window.cpp (703) createTrackbar UI/Trackbar(sp@img): Using 'value' pointer is unsafe and deprecated. Use NULL as value pointer. To fetch trackbar value setup callback.
[ WARN:0] global /tmp/pip-req-build-afu9cjzs/opencv/modules/highgui/src/window.cpp (703) createTrackbar UI/Trackbar(sr@img): Using 'value' pointer is unsafe and deprecated. Use NULL as value pointer. To fetch trackbar value setup callback.
[ WARN:0] global /tmp/pip-req-build-afu9cjzs/opencv/modules/highgui/src/window.cpp (703) createTrackbar UI/Trackbar(lv@img): Using 'value' pointer is unsafe and deprecated. Use NULL as value pointer. To fetch trackbar value setup callback.
